In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(DESeq2)
library(magrittr)
library(SummarizedExperiment)

IN_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/input/"
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/output/"

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading require

In [3]:
%%R
dds <- readRDS(paste(OUT_DIR,"dds0.03vs0_time_course.Rds",sep=""))
print(dds)

class: DESeqDataSet 
dim: 9297 91 
metadata(1): version
assays(1): counts
rownames(9297): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(0):
colnames(91): SRR7817611 SRR7817612 ... SRR4317611 SRR4317612
colData names(2): Sample Dose


In [4]:
%%R
vst.counts <- DESeq2::vst(dds,
                          blind = FALSE,
                          fitType = "local")
print(vst.counts)

class: DESeqTransform 
dim: 9297 91 
metadata(1): version
assays(1): ''
rownames(9297): ENSMUSG00000000001 ENSMUSG00000000028 ...
  ENSMUSG00000118462 ENSMUSG00000118541
rowData names(4): baseMean baseVar allZero dispFit
colnames(91): SRR7817611 SRR7817612 ... SRR4317611 SRR4317612
colData names(3): Sample Dose sizeFactor


In [5]:
%%R
saveRDS(vst.counts,
        paste(OUT_DIR,"vst_counts0.03vs0_time_course.Rds",sep=""))

In [6]:
%%R
ensembl2rxns.df <- read.table(paste(IN_DIR,"Ensembl2ReactomeReactions.txt",sep=""),
                              sep="\t")
head(ensembl2rxns.df, n=5)

            V1            V2                                   V3
1 DDB_G0267374 R-DDI-6809003 https://reactome.org/PathwayBrowser/
2 DDB_G0267374  R-DDI-879459 https://reactome.org/PathwayBrowser/
3 DDB_G0267374  R-DDI-994148 https://reactome.org/PathwayBrowser/
4 DDB_G0267378 R-DDI-1237042 https://reactome.org/PathwayBrowser/
5 DDB_G0267378 R-DDI-1247649 https://reactome.org/PathwayBrowser/


In [ ]:
%%R
reactome_ensembl_ids <- readRDS(paste(OUT_DIR,"reactome_ensembl_ids0.03vs0.Rds",sep=""))
print(reactome_ensembl_ids)

In [8]:
%%R
rxn2ensembls.nls <- list()
rxns_w_tcdd_ensembls.df <- ensembl2rxns.df %>% dplyr::filter(V1 %in% reactome_ensembl_ids)
print(head(rxns_w_tcdd_ensembls.df))
print(dim(rxns_w_tcdd_ensembls.df))

                  V1           V2                                   V3
1 ENSMUSG00000000001 R-MMU-170666 https://reactome.org/PathwayBrowser/
2 ENSMUSG00000000001 R-MMU-170671 https://reactome.org/PathwayBrowser/
3 ENSMUSG00000000001 R-MMU-170674 https://reactome.org/PathwayBrowser/
4 ENSMUSG00000000001 R-MMU-170686 https://reactome.org/PathwayBrowser/
5 ENSMUSG00000000001 R-MMU-380073 https://reactome.org/PathwayBrowser/
6 ENSMUSG00000000001 R-MMU-392187 https://reactome.org/PathwayBrowser/
[1] 81047     3


In [9]:
%%R
rxns_w_tcdd_ensembls.df$V1 <- as.character(rxns_w_tcdd_ensembls.df$V1)
rxns_w_tcdd_ensembls.df$V2 <- as.character(rxns_w_tcdd_ensembls.df$V2)


for(i in 1:nrow(rxns_w_tcdd_ensembls.df)){
  ens_id <- rxns_w_tcdd_ensembls.df$V1[i]
  rxn_id <- rxns_w_tcdd_ensembls.df$V2[i]
#     print(names(rxn2ensembls.nls))

  ensembl_list_for_rxn_id <- rxn2ensembls.nls[[rxn_id]]
  if(is.null(ensembl_list_for_rxn_id)){
#     print("null")
    ensembl_list_for_rxn_id <- c(ens_id)
  }
  rxn2ensembls.nls[[rxn_id]] <- c(ensembl_list_for_rxn_id,ens_id) %>% unique()
}
print(length(rxn2ensembls.nls))
saveRDS(rxn2ensembls.nls,
        paste(OUT_DIR,"rxn2ensembls_nls0.03vs0_time_course.Rds",sep=""))

[1] 9825
